# Linear solvers in the TT format

This tutorial addresses solving multilinear systems $\mathsf{Ax}=\mathsf{b}$ in the TT format.

Imports:

In [1]:

import torch as tn
import datetime
try: 
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT
    import torchtt as tntt

### Small example

A random tensor operator $\mathsf{A}$ is created in the TT format. We create a random right-hand side $\mathsf{b} = \mathsf{Ax}$, where $\mathsf{x}$ is a random tensor in the TT format. This way the solution of $\mathsf{Ax}=\mathsf{b}$ is known and we can compare it as a reference.
This works only for small random tensors.

In [2]:
A = tntt.random([(4,4),(5,5),(6,6)],[1,2,3,1]) 
x = tntt.random([4,5,6],[1,2,3,1])
b = A @ x

Solve the multilinear system $\mathsf{Ax}=\mathsf{b}$ using the method torchtt.solvers.amen_solve().


In [3]:
xs = tntt.solvers.amen_solve(A,b, x0 = b, eps = 1e-7)

The relative residual norm and the relative error of the solution are reported:

In [4]:
print(xs)
print('Relative residual error ',(A@xs-b).norm()/b.norm())
print('Relative error of the solution  ',(xs-x).norm()/x.norm())


TT with sizes and ranks:
N = [4, 5, 6]
R = [1, 2, 3, 1]

Device: cpu, dtype: torch.float64
#entries 56 compression 0.4666666666666667

Relative residual error  tensor(4.8851e-12, dtype=torch.float64)
Relative error of the solution   tensor(1.4532e-11, dtype=torch.float64)


### Finite differences

We now solve the problem $\Delta u = 1$ in $[0,1]^d$ with $ u = 0 $ on the entire boundary using finite differences.
First, set the size of the problem (n is the mode size and d is the number of dimensions):

In [5]:
dtype = tn.float64 
n =  256
d = 8

Create the finite differences matrix corresponding to the problem. The operator is constructed directly in the TT format as it follows

In [6]:
L1d = -2*tn.eye(n, dtype = dtype)+tn.diag(tn.ones(n-1,dtype = dtype),-1)+tn.diag(tn.ones(n-1,dtype = dtype),1)
L1d[0,1] = 0
L1d[-1,-2] = 0
L1d /= (n-1)
L1d = tntt.TT(L1d, [(n,n)])

L_tt = tntt.zeros([(n,n)]*d)
for i in range(1,d-1):
    L_tt = L_tt+tntt.eye([n]*i)**L1d**tntt.eye([n]*(d-2))
L_tt = L_tt + L1d**tntt.eye([n]*(d-1)) +  tntt.eye([n]*(d-1))**L1d
L_tt = L_tt.round(1e-14)

The right hand site of the finite difference system is also computed in the TT format

In [7]:
b1d = tn.ones(n, dtype=dtype)
b1d[0] = 0
b1d[-1] = 0
b1d = tntt.TT(b1d)
b_tt = b1d
for i in range(d-1):
    b_tt = b_tt**b1d

Solve the system

In [8]:
time = datetime.datetime.now()
x = tntt.solvers.amen_solve(L_tt, b_tt ,x0 = b_tt, nswp = 20, eps = 1e-8, verbose = True)
time = datetime.datetime.now() - time
print('Relative residual: ',(L_tt@x-b_tt).norm()/b_tt.norm())
print('Solver time: ',time)


Starting sweep 0 ...
	Core 0
		Choosing direct solver (local size 256)....
		dx = 1.00022, res_now = 4.00336e-15, res_old = 1.00025
	Core 1
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 1.99174e-07 (from 1.12305e-09)
		Time needed  0:00:00.363313
		dx = 0.162252, res_now = 1.99174e-07, res_old = 1.57407
	Core 2
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 4.73222e-07 (from 9.83027e-10)
		Time needed  0:00:00.478912
		dx = 0.178784, res_now = 4.73222e-07, res_old = 1.79829
	Core 3
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 1.58832e-06 (from 8.41988e-10)
		Time needed  0:00:00.555476
		dx = 0.201373, res_now = 1.58832e-06, res_old = 2.09952
	Core 4
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 7.85965e-06 (from 6.99841e-10)
		T

Display the structure of the TT

In [9]:
print(x)

TT with sizes and ranks:
N = [256, 256, 256, 256, 256, 256, 256, 256]
R = [1, 18, 20, 20, 20, 19, 19, 18, 1]

Device: cpu, dtype: torch.float64
#entries 583424 compression 3.162747841400915e-14



Try one more time on the GPU (if available).

In [11]:
if tn.cuda.is_available():
    cuda_dev = 'cuda:0'
    time = datetime.datetime.now()
    x = tntt.solvers.amen_solve(L_tt.to(cuda_dev), b_tt.to(cuda_dev) ,x0 = b_tt.to(cuda_dev), nswp = 20, eps = 1e-8, verbose = True, preconditioner=None)
    time = datetime.datetime.now() - time
    x = x.cpu()
    print('Relative residual: ',(L_tt@x-b_tt).norm()/b_tt.norm())
    print('Solver time: ',time)
else:
    print('GPU not available...')


Starting sweep 0 ...
	Core 0
		Choosing direct solver (local size 256)....
		dx = 1.00022, res_now = 3.27236e-15, res_old = 1.00025
	Core 1
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 2.69736e-07 (from 1.12282e-09)
		Time needed  0:00:00.356805
		dx = 0.162221, res_now = 2.69736e-07, res_old = 1.5744
	Core 2
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 4.66863e-07 (from 9.82968e-10)
		Time needed  0:00:00.349299
		dx = 0.17877, res_now = 4.66863e-07, res_old = 1.7984
	Core 3
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 1.35812e-06 (from 8.41961e-10)
		Time needed  0:00:00.352803
		dx = 0.201367, res_now = 1.35812e-06, res_old = 2.09958
	Core 4
		Choosing iterative solver GMRES (local size 1280)....
		Finished with flag 0 after 80 iterations with relres 6.00668e-06 (from 6.99868e-10)
		Time